Data has already been downloaded, can be found here:
```
/data/gersbachlab/srm79/seq_data/Pilot_CD2_scRNAseq_screen/
```

In [ ]:
# %load /data/reddylab/Alex/tmp/cell_ranger.clean.csv
Lane,Sample,Index
*,Feature_Barcode_Lib_1,AGTATCTGCA
*,Feature_Barcode_Lib_2,TCTATGAGTG
*,Guide_Lib_1,GAACTGAGCG
*,Guide_Lib_2,AGGTCAGATA
*,Targeted_GEX_Lib_1,GTAACATGCG
*,Targeted_GEX_Lib_2,GTGGATCAAA


In [15]:
%%bash
mkdir -p /data/reddylab/Alex/collab/20210526_Sean/logs
mkdir -p /data/reddylab/Alex/collab/20210526_Sean/scripts
mkdir -p /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata
mkdir -p /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/fastq
mkdir -p /data/reddylab/Alex/collab/20210526_Sean/results/sc_rna_seq

In [18]:
%%writefile /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/Pilot_CD2_scRNAseq_screen.samplesheet.csv
[Header],,,,,,,,,
IEMFileVersion,4,,,,,,,,
Investigator Name,Sean McCutcheon,,,,,,,,
Experiment Name,Pilot_CD2_scRNAseq_screen,,,,,,,,
Date,4/20/21,,,,,,,,
Workflow,GenerateFASTQ,,,,,,,,
Application,FASTQ Only,,,,,,,,
Assay,Nextera,,,,,,,,
Description,Pilot_CD2_scRNAseq_screen,,,,,,,,
Chemistry,Amplicon,,,,,,,,
,,,,,,,,,
[Reads],,,,,,,,,
26,,,,,,,,,
90,,,,,,,,,
,,,,,,,,,
[Settings],,,,,,,,,
ReverseComplement,0,,,,,,,,
,,,,,,,,,
[Data],,,,,,,,,
Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Description
Feature_Barcode_Lib_1,,,,SI-GA-F1_1,AGTATCTGCA,i5,,,
Feature_Barcode_Lib_2,,,,SI-GA-F1_2,TCTATGAGTG,i5,,,
Guide_Lib_1,,,,SI-GA-F1_3,GAACTGAGCG,i5,,,
Guide_Lib_2,,,,SI-GA-F1_4,AGGTCAGATA,i5,,,
Targeted_GEX_Lib_1,,,,SI-GA-F1_5,GTAACATGCG,i5,,,
Targeted_GEX_Lib_2,,,,SI-GA-F1_6,GTGGATCAAA,i5,,,


Overwriting /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/Pilot_CD2_scRNAseq_screen.samplesheet.csv


In [11]:
indices = !cut -f3 -d, '/data/reddylab/Alex/tmp/cell_ranger.clean.csv' | tail -n+2

In [1]:
samples = !cut -f2 -d, '/data/reddylab/Alex/tmp/cell_ranger.clean.csv' | tail -n+2

In [3]:
samples

['Feature_Barcode_Lib_1',
 'Feature_Barcode_Lib_2',
 'Guide_Lib_1',
 'Guide_Lib_2',
 'Targeted_GEX_Lib_1',
 'Targeted_GEX_Lib_2']

In [19]:
%%bash
module load bcl2fastq2/v2.20.0.422-gcb01
sbatch \
    --partition new,all \
    --mem 64G \
    --cpus-per-task 16 \
    -o /data/reddylab/Alex/collab/20210526_Sean/logs/bcl2fastq.Pilot_CD2_scRNAseq_screen.10x_demultiplex.log \
    <<'EOF'
#!/bin/bash
bcl2fastq \
    -R /data/gersbachlab/srm79/seq_data/Pilot_CD2_scRNAseq_screen/ \
    -p 16 \
    --output-dir /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/fastq \
    --sample-sheet /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/Pilot_CD2_scRNAseq_screen.samplesheet.csv
    
EOF


Submitted batch job 25432709


>reminder that read 2 for the guide libs with need to be reverse complemented before alignment

In [16]:
%%writefile /data/reddylab/Alex/collab/20210526_Sean/scripts/revcomp.fastq.py
#!/usr/bin/env python
import sys

def revcomp(s):
    rev = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'N': 'N'}
    return ''.join([rev[c] for c in s[::-1]])

def main():
    header = None
    seq = None
    qualities = None
    if len(sys.argv)>1:
        inf = open(sys.argv[1], 'r') 
    else:
        inf = sys.stdin
    with inf as ifile:
        for l_ix, l in enumerate(ifile):
            if l_ix % 4 == 0:
                if header:
                    print("\n".join([header, seq, "+", qualities]))
                header = l.rstrip()
            elif l_ix % 4 == 1:
                seq = revcomp(l.rstrip())
            elif l_ix % 4 == 3:
                qualities = l.rstrip()[::-1]
        else:
            print("\n".join([header, seq, "+", qualities]))

if __name__ == '__main__':
    main()

Overwriting /data//reddylab/Alex/collab/20210526_Sean/scripts/revcomp.fastq.py


In [18]:
%%writefile /data/reddylab/Alex/collab/20210526_Sean/scripts/revcomp.fasta.py
#!/usr/bin/env python
import sys

def revcomp(s):
    rev = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'N': 'N'}
    return ''.join([rev[c] for c in s[::-1]])

def main():
    header = None
    seq = None
    if len(sys.argv)>1:
        inf = open(sys.argv[1], 'r') 
    else:
        inf = sys.stdin
    with inf as ifile:
        for l_ix, l in enumerate(ifile):
            if l_ix % 2 == 0:
                if header:
                    print("\n".join([header, seq]))
                header = l.rstrip()
            elif l_ix % 2 == 1:
                seq = revcomp(l.rstrip())
        else:
            print("\n".join([header, seq]))

if __name__ == '__main__':
    main()

Overwriting /data/reddylab/Alex/collab/20210526_Sean/scripts/revcomp.fasta.py


In [ ]:
%%bash
for ii in /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/fastq/Guide_Lib_[12]*R2*; do gzip -dc $ii | python /data//reddylab/Alex/collab/20210526_Sean/scripts/revcomp.fastq.py  | gzip -c > ${ii/Guide_Lib_/Guide_Lib_revcomp_} ; done

In [20]:
%%bash
cd /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/fastq/
# cp Guide_Lib_[12]*R2* /data/reddylab/Alex/tmp/
rename Guide_Lib_revcomp Guide_Lib Guide_Lib_revcomp*

We need to break apart  

In [7]:
%%bash
mkdir -p /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/count_2/
cd /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/count_2/
#-15,17-20%10
SAMPLES=($(cut -f2 -d, '/data/reddylab/Alex/tmp/cell_ranger.clean.csv' | tail -n+2))
# 1-$((${#SAMPLES[@]}-1))
sbatch \
    --array=0 \
    -p new,all \
    -o /data/reddylab/Alex/collab/20210526_Sean/logs/sgrnas_10x_processing.%a.out \
    --mem=64G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
SAMPLES=($(cut -f2 -d, '/data/reddylab/Alex/tmp/cell_ranger.clean.csv' | tail -n+2))
SAMPLE=${SAMPLES[${SLURM_ARRAY_TASK_ID}]}
/data/reddylab/software/cellranger/cellranger-6.0.1/bin/cellranger count \
    --id=${SAMPLE} \
    --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
    --feature-ref=/data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/feature_reference.csv \
    --libraries=/data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/feature_barcode_lib.csv \
    --localcores=24 \
    --localmem=64 \
    --chemistry=SC5P-R2
EOF

Submitted batch job 25434153


In [ ]:
/data/reddylab/software/cellranger/bamtofastq/bamtofastq_linux --nthreads="$NTHREADS" pilot_highmoi_screen.1_SI_GA_G1.bam bam_pilot_scrna_1

In [ ]:
# %load /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/CD2_guide_feature_reference.csv
id,name,read,pattern,sequence,feature_type,target_gene_id,target_gene_name
CD2_g1,CD2_g1,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGGAGACTGCACCTATAAAAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g2,CD2_g2,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGAGCTTAACCACGTGCCTCT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g3,CD2_g3,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGCTCTTCATGCAAAAGGAAC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g4,CD2_g4,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGTTTCCAAGGCATTCGTAAT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g5,CD2_g5,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTTAGGGGTTGGTTTCCTCTT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g6,CD2_g6,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGATTCTGAGCTCTTCATGC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g7,CD2_g7,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGTTTTCAAATGAGTGATGAT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g8,CD2_g8,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTTTTGATTCTGAGCTCTTCA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g9,CD2_g9,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTTGATTCTGAGCTCTTCAT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g10,CD2_g10,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGAAAACTAGGAATGTCCAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g11,CD2_g11,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GAGCTTAACCACGTGCCTCTC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g12,CD2_g12,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGCTTTTTATAGGTGCAGTC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g13,CD2_g13,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTGCATGAAGAGCTCAGAAT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g14,CD2_g14,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTTTCCCTGAAAGTGACTCTC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g15,CD2_g15,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGATGAGCTTTCCATGTAAAT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g16,CD2_g16,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTTGATGTCCTGACCCAAGGC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g17,CD2_g17,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGGGTGTGGACTCCACCAGTC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g18,CD2_g18,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAGATGAGCTTTCCATGTAAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g19,CD2_g19,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGACTCTTATGCTTACCTTTG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g20,CD2_g20,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATGCTTACCTTTGGAAGAAAC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g21,CD2_g21,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGGTAAGCATAAGAGTCAAAG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g22,CD2_g22,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGTGTGGACTCCACCAGTCTC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g23,CD2_g23,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GAAAGTGACTCTCAGTAACTC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g24,CD2_g24,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAGAGATTACGAATGCCTTGG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g25,CD2_g25,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTTTCCAAGGCATTCGTAATC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g26,CD2_g26,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CCAGTCTCACTTCAGTTCCTT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g27,CD2_g27,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAGGTAAGCATAAGAGTCAAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g28,CD2_g28,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TCTGATTTTCAATGTTTCTTC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g29,CD2_g29,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AATCTCTTTGGAGACTGCACC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g30,CD2_g30,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTCAGTTCCTTTTGCATGAAG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g31,CD2_g31,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GCAAAAGGAACTGAAGTGAGA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g32,CD2_g32,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGTAACTCTTTTGCTTTTTAT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g33,CD2_g33,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGCATTCGTAATCTCTTTGGA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g34,CD2_g34,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AATCAAAAGAGGAAACCAACC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g35,CD2_g35,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TAACTCTTTTGCTTTTTATAG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g36,CD2_g36,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGCACCCCAGGTTTCCAAGGC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g37,CD2_g37,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGAAAGTGACTCTCAGTAACT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g38,CD2_g38,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTAAATTTGTAGCCAGCTTCC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g39,CD2_g39,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACCCAGCTTTCCCTGAAAGTG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g40,CD2_g40,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTTTTCAAATGAGTGATGATA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g41,CD2_g41,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CAGTCTCACTTCAGTTCCTTT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g42,CD2_g42,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CCAAAGGTAAGCATAAGAGTC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g43,CD2_g43,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTACATCTTTTACAGTAACA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g44,CD2_g44,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TACATCTTTTACAGTAACATA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g45,CD2_g45,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGGCATTCGTAATCTCTTTGG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g46,CD2_g46,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CCCAGCTTTCCCTGAAAGTGA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g47,CD2_g47,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATGAGTGATGATATTGACGAT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g48,CD2_g48,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTCCTTTTGCATGAAGAGCTC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g49,CD2_g49,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACCCCGAGAGCTTAACCACGT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g50,CD2_g50,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTAAATTTGTAGCCAGCTTC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g51,CD2_g51,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGAGATTACGAATGCCTTGGA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g52,CD2_g52,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAAGTGACTCTCAGTAACTCT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g53,CD2_g53,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CATGCAAAAGGAACTGAAGTG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g54,CD2_g54,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTGATTTTCAATGTTTCTTCC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g55,CD2_g55,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGAAAGCTGGGTTGGGACTTC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g56,CD2_g56,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATTTTCAATGTTTCTTCCAAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g57,CD2_g57,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACCCAAGGCACCCCAGGTTTC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g58,CD2_g58,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTTTCTTCCAAAGGTAAGCA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g59,CD2_g59,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGCATGAAGAGCTCAGAATC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g60,CD2_g60,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTAGGTATACTGTCCCTAACA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g61,CD2_g61,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTATTGGCTTGTGAACATTTA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g62,CD2_g62,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGTAAACAAGAAGCCAAATT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g63,CD2_g63,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTGGTCTTGTTAGGGACAGT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g64,CD2_g64,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GCTGCATTTTAATTCTCACAC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g65,CD2_g65,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CCCAATTTTTTTGTGTGAGAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g66,CD2_g66,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTGATGTGCTCCTTTGCCAAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g67,CD2_g67,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGAAAGAGAAAAGGAACAGAG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g68,CD2_g68,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATGTGCTCCTTTGCCAAAGCA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g69,CD2_g69,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGCCAAAGCAGATGTGTTTA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g70,CD2_g70,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACACAAAAAAATTGGGAAGGA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g71,CD2_g71,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAACACATCTGCTTTGGCAAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g72,CD2_g72,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGTGATCTACTTAGCCTCTCT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g73,CD2_g73,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CCACACAGCCTTTTATCCACT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g74,CD2_g74,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGCCAAAGCAGATGTGTTTAT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g75,CD2_g75,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTCTCTTTCACTGAGATGAGA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g76,CD2_g76,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACAAGACCACATAGAAATATA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g77,CD2_g77,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTTCCTTTTCTCTTTCACTG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g78,CD2_g78,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGAGATGAGAAAACCTATCCT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g79,CD2_g79,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTCACTGAGATGAGAAAACC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g80,CD2_g80,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTTTACAAAAGAGTGATCCTT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g81,CD2_g81,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTCTCTTTCACTGAGATGAG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g82,CD2_g82,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTCCTTTTCTCTTTCACTGAG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g83,CD2_g83,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AACAAGACCACATAGAAATAT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g84,CD2_g84,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATTTTTTTGTGTGAGAATTAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g85,CD2_g85,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GCTCTCTGTGTATGTGTATTA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g86,CD2_g86,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACAAAAAAATTGGGAAGGATA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g87,CD2_g87,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTGTATGTGTATTATGTTTT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g88,CD2_g88,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TCACTGAGATGAGAAAACCTA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g89,CD2_g89,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTTTTTGTGTGAGAATTAAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g90,CD2_g90,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTCAGTGAAAGAGAAAAGGAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g91,CD2_g91,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAGCCAATAGCAGAATGCAAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g92,CD2_g92,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATCTCAGTGAAAGAGAAAAGG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g93,CD2_g93,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTATGTGTATTATGTTTTAT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g94,CD2_g94,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AATTTTTTTGTGTGAGAATTA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g95,CD2_g95,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATCATCGGTCTTCAGATGCTT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g96,CD2_g96,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGTAAGTGTTCATTCCCTTAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g97,CD2_g97,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGGGGGCTATACAGGAAACCA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g98,CD2_g98,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTATAGCCCCCCTGGAGGCAG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g99,CD2_g99,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTCTGAATTGTGCAATCTTTT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g100,CD2_g100,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTAATTGCTTTATTTCAGTGT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g101,CD2_g101,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AATTGTGCAATCTTTTTCTTG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g102,CD2_g102,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACTGAAATAAAGCAATTAAGG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g103,CD2_g103,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTATATCGTCAATATCATCA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g104,CD2_g104,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATTTTATATCGTCAATATCAT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g105,CD2_g105,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TCTCTTTTCTGAATTGTGCAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g106,CD2_g106,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTTGCCAAATAGCACCCACAC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g107,CD2_g107,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATATTCCCCAGCGCTGACCTC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g108,CD2_g108,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATGCTTAATTTTCAGAGTTCC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g109,CD2_g109,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TATAGCCCCCCTGGAGGCAGA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g110,CD2_g110,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAAATTAAGCATCTGAAGACC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g111,CD2_g111,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TAATTGCTTTATTTCAGTGTG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g112,CD2_g112,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACTTGCCAAATAGCACCCACA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g113,CD2_g113,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTGGAGGCAGAGGTCAGCGCT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g114,CD2_g114,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AATTGCTTTATTTCAGTGTGG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g115,CD2_g115,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGAAAGTCTCTTTCTCTTTTC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g116,CD2_g116,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGGGGCTATACAGGAAACCAG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g117,CD2_g117,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTTCTGAATTGTGCAATCTT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g118,CD2_g118,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CAGCGCTGACCTCTGCCTCCA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g119,CD2_g119,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTTATATCGTCAATATCATC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g120,CD2_g120,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CCTCCAGGGGGGCTATACAGG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g121,CD2_g121,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTGGGTGCTATTTGGCAAGT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g122,CD2_g122,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TCCCTTAATTGCTTTATTTCA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g123,CD2_g123,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GAAAGTCTCTTTCTCTTTTCT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g124,CD2_g124,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAAAGAGAAAGAGACTTTCAA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g125,CD2_g125,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGCGCTGACCTCTGCCTCCAG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g126,CD2_g126,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATCAAATATTTTTTCCAACAC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g127,CD2_g127,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGTTCCATTTTTAAATAGCTT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g128,CD2_g128,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACAAGAAAAAGATTGCACAAT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g129,CD2_g129,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GCGCTGACCTCTGCCTCCAGG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g130,CD2_g130,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGAAAAAGATACATATAAGCT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g131,CD2_g131,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTCTGAAGTTTTTTCCCATTT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g132,CD2_g132,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTTTAAATAGCTTATATGTA,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g133,CD2_g133,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGGAGGCAGAGGTCAGCGCTG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g134,CD2_g134,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTGTATCATATATTGATACC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g135,CD2_g135,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGAGGCAGAGGTCAGCGCTGG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g136,CD2_g136,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GAGGCAGAGGTCAGCGCTGGG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g137,CD2_g137,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CGCTGACCTCTGCCTCCAGGG,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g138,CD2_g138,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTTAAATAGCTTATATGTAT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g139,CD2_g139,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGTATCATATATTGATACCT,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g140,CD2_g140,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CACATTTTTTCCTTTTGTATC,CRISPR Guide Capture,ENSG00000116824,CD2
CD2_g141,CD2_g141,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACACATTTTTTCCTTTTGTAT,CRISPR Guide Capture,ENSG00000116824,CD2
NT_1,NT_1,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TACAACCGTCCTTCCCATAAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_2,NT_2,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTAAGTTACTCGCTAAAGCC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_3,NT_3,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TATCACATATCATATCAAATA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_4,NT_4,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGCAAATGCATGTTAATACTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_5,NT_5,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGTGCAGAAGCTAGAGTTATC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_6,NT_6,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CGTATGCGTAAGATGTTAGAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_7,NT_7,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAGGAATCTAATTGTATTTTC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_8,NT_8,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTCTTTGCTCATATAGCAAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_9,NT_9,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACACGGTAACCTTCAGCTTGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_10,NT_10,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GATGTATTCTCCATTGCGTAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_11,NT_11,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GCTGAATGGATATTTATGGCC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_12,NT_12,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGGGAGCATCGTCCGACTGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_13,NT_13,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGGCCTTTATTCTAGCTAGTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_14,NT_14,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CCGGGCTCCTGTATAAAATTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_15,NT_15,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACCAGATTTATACTACAAACG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_16,NT_16,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGGAAATTCAGTCAAAAATTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_17,NT_17,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGCCCTAGTAATTATAGAACG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_18,NT_18,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTGCAATTTTAAATATTATGG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_19,NT_19,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTCCCGTACTGAGTGTTAGGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_20,NT_20,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TAAGACTGCGTCCAACTGATG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_21,NT_21,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTATACGTATGTGTGTACAAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_22,NT_22,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATGTCTTTCTATCTGAAATGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_23,NT_23,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGTCCGTCTTTTCAAGGCCGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_24,NT_24,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CGTCACATCATCTAGAGACTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_25,NT_25,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATGTAAATCTACTGTTTAGTG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_26,NT_26,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTAAACCTTTTACAACATCAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_27,NT_27,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAAACTTTACATTTAATTTTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_28,NT_28,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTATCCCCCGTCTTGAAACA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_29,NT_29,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTCTTTTGAGGCGGTGGATC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_30,NT_30,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATTTTTACTTGCTCGTACCTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_31,NT_31,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GATCTCAGAATTCCGTTGTGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_32,NT_32,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGACTGTTGTTATATTACCA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_33,NT_33,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTAAACCTACACAAACGCCGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_34,NT_34,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATGGTTAGAGGCGCTTAGCTG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_35,NT_35,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TATAGCTGTTTATAGGATACA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_36,NT_36,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGGGATTATCTTCACCCTGTC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_37,NT_37,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACAAAGAAACATGAACGCATT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_38,NT_38,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTAGCCCCCTGGTAGCCTAAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_39,NT_39,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTGACTCTCTGTATGGCTGCG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_40,NT_40,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGGTTAATGTAATATGCTAAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_41,NT_41,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TAAACCGGTCGTGTCACTAGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_42,NT_42,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATGATGTCAGGAGGAACTCAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_43,NT_43,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TATCGATAATATTCTTCTTAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_44,NT_44,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGGAGTAAAACGGATTACTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_45,NT_45,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGGGCATCGACGATCGGAGG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_46,NT_46,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGATCAAACATATGACCTGCG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_47,NT_47,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTTGGTAAAAGCTCCATTGTG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_48,NT_48,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CGCCCATGTTCAAGGATGTCG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_49,NT_49,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTTCCCAATTTGGCATAACAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_50,NT_50,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CGCGTTGCTTAACATTCCATT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_51,NT_51,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTATCAATACAGCGCAGAAC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_52,NT_52,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTCTGATTAACTAGGGAAATC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_53,NT_53,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CGCCAGTATGTCTAACCTTGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_54,NT_54,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATATCTTTCTACGTTAGACTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_55,NT_55,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CATCATGTGTTGACGGTAGCC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_56,NT_56,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGAGATTCTAAATTTCAAATA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_57,NT_57,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATTACCCTAACCTTCAATTTC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_58,NT_58,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTATGGAAAGGTATAATAATA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_59,NT_59,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGCCGTGGTATTCTCGCTGTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_60,NT_60,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAGACTTTATGGGTCACTGTG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_61,NT_61,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACAGTGCATGATAGGAAGAAG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_62,NT_62,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTCGTAGGGACCGATATGAG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_63,NT_63,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CACTTCTCTGCGGTCATAACC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_64,NT_64,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATCCATTTAGGTCAGGGATTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_65,NT_65,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTGCGGACCAGAGATGCTTTG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_66,NT_66,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTCAGGCTCTCATAATTAAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_67,NT_67,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTAATTTTTTCAAAAGTAGCT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_68,NT_68,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TCTCGATTTAGTTGTGTTTTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_69,NT_69,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GCGTTCATCCTCACCATACAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_70,NT_70,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTGAACCTAATAGGCATGAAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_71,NT_71,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTAAGACTTCACTAGTCTCGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_72,NT_72,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGAGGCTTTGGTTAAGTCCGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_73,NT_73,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGGAGCTTTATTATTTTCCCT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_74,NT_74,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGCCAGAGATTAGGAATAATT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_75,NT_75,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGTCTGTTGTATTAAACCATC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_76,NT_76,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CATACGATTTTCCTCCAAGGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_77,NT_77,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTGAGCGTTTGCGACTCGTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_78,NT_78,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAGCAACAGGAACAGCCATTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_79,NT_79,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTACTCAATTGTAGTCTCGTG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_80,NT_80,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GCCTCGCCTCTCGGGATAATC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_81,NT_81,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAAATGAGAGAAGATCACGAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_82,NT_82,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTCAATACGAGTTGAGAAATG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_83,NT_83,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GCGCATCTTCCTATTTAACTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_84,NT_84,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACAATTCTATGCGACACTTCA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_85,NT_85,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACGTTACTCCGATTTATTGAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_86,NT_86,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATAGTTTTGGCCGATCTTGAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_87,NT_87,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TAAGACCAACTGTGGTTGAGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_88,NT_88,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGCTCACGTAACGGGAGGGGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_89,NT_89,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGCGAAGTAGCATTTTCCTCC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_90,NT_90,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AATCGAAAAAACAAGCAAGCT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_91,NT_91,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TCCAAAAATGAATTGGCCCGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_92,NT_92,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTATAATATATACAAAACCC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_93,NT_93,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CCCATTTCTGCACCGTTCTTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_94,NT_94,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGATCCGTTTGTAGTACGCTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_95,NT_95,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CGCATTGCTTGACGACCACTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_96,NT_96,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTCGTTAAGACCTTTCGAGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_97,NT_97,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTGCGTGCAGACTTGTTGTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_98,NT_98,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGCCTCCCTCACCTATTTAG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_99,NT_99,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGACGGGTGTTCTAGACCTG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_100,NT_100,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTCTTTTTCCTAGGAGCTTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_101,NT_101,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTAGTGAATCTTTTTGGCAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_102,NT_102,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTCATCCGTAACTGGATCCAC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_103,NT_103,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TACAATCTTCACTGTGCTAAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_104,NT_104,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTACGCTTTAATTTGATCAAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_105,NT_105,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTCCAAAGCTTCTCTAAATGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_106,NT_106,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATTTCTAGCGCACCCCATAAG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_107,NT_107,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTCCATTGTTAGCTAAATCA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_108,NT_108,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TCCCATCGCTACCAATAAACT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_109,NT_109,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGATATAACATCCGGATCATG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_110,NT_110,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATACTGCCGTGTTTTTAGAAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_111,NT_111,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CAACAGTCTTAGGTAGAGGTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_112,NT_112,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACCGTATTTCCTCGCTCGTAC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_113,NT_113,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CGTCTGACTAGAAACGGTCAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_114,NT_114,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATGAACTCCCTTGTTACACAG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_115,NT_115,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATTACTGGAACACCTAGTACT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_116,NT_116,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CAACAGGCGTGACACGGTCGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_117,NT_117,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACGCAACCTCTTTAGGCATCG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_118,NT_118,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGCAAAAACCGCGTTAATATT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_119,NT_119,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGAAATAATCTAGTAGCCAAC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_120,NT_120,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATACACTTGTTCGAATTCCAG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_121,NT_121,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GATCTCTCCTCAGTTCTTTGA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_122,NT_122,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGTGCAATACCACCCGCACAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_123,NT_123,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTCTCTGATAGGGCGACTTC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_124,NT_124,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GCAATTATCTCGTGCATAATC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_125,NT_125,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CGCGAGGTTTGTATCAGTCTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_126,NT_126,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTTAGGATTACTGGCTTCACA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_127,NT_127,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CAGGATTTTTTAGGTCTTGAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_128,NT_128,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATGTGCATAAAACGCTATTTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_129,NT_129,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTCATGGTTAAGTTTTTATTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_130,NT_130,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACTTATTATATCAGTTGTAGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_131,NT_131,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTTAGAACTTTCCAGCTCTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_132,NT_132,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGGGTATCATTCCGATCATAG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_133,NT_133,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTGTTTTTCTTTTAAAATTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_134,NT_134,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGTCTATATAACTGGTATCTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_135,NT_135,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAACACTGTCACAGAATATAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_136,NT_136,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TAAGCATGGAAAGTATATGTG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_137,NT_137,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GAGGAGTCCATATTACAACTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_138,NT_138,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CCAATATATGGAAGAGAGAGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_139,NT_139,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTCCAATCTCTAGGTGTCTG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_140,NT_140,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACATTATGCCGTGAGGCCTCC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_141,NT_141,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACGGCGCTACATTTAAATATA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_142,NT_142,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGGGTCCGTACAGTAGATAAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_143,NT_143,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATATCCCGAATCTAATCCGAC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_144,NT_144,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CGATAGACCTTCAAAGAAATC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_145,NT_145,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GAGTCGAAAATCCTCTACCGG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_146,NT_146,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CCTAGCGAGTGTACGCATCGG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_147,NT_147,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TAGACTTCTTTTGTACGTAGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_148,NT_148,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATGACTATTGGATGCACCGCC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_149,NT_149,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CCGTTAATAGGAATAATGCTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_150,NT_150,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTAGTTATTTCATATCCGAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_151,NT_151,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTATGTACTAAACCGTATCTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_152,NT_152,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATCGGTATATACGGTTAGTGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_153,NT_153,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTTATTCGCTGGGAATTTCGA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_154,NT_154,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CCTTTAGCTCCTGAATATATT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_155,NT_155,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTCTAATCGGTGCATTATTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_156,NT_156,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CGTCTTCACAGCTTAAATGTC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_157,NT_157,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAGCGTTCAAGCAGAAGTAGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_158,NT_158,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGGACGTCCCTAGTGCTGATC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_159,NT_159,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CAGTCAACCGCTTGCCCACAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_160,NT_160,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATGGATTATGGTGTTGTGCGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_161,NT_161,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTGATTACAACCCTAAGATG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_162,NT_162,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TCCCTGATATAAGTTTCTACG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_163,NT_163,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGAACATTGCCCTTAGTTCCC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_164,NT_164,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTCTTCCCGGTGGCATCGTC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_165,NT_165,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GCTTATGCGATGTGTCTCGAG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_166,NT_166,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTTTTGCATATTGAATCACTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_167,NT_167,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CAACGGTCTTGTGGATCGCAC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_168,NT_168,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AATGATAGGCAAGCTACAAAC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_169,NT_169,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TAGCTCTAACTCTCAGTTACA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_170,NT_170,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATCACGATTAATAATATACTC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_171,NT_171,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTACTTTAAAATGGCTCCTTC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_172,NT_172,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGTTCTAAACTTATAGGATAC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_173,NT_173,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TCTGTACGATTTTGTGTGGTC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_174,NT_174,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAGTGTTATCTTTAGAAAATT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_175,NT_175,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGACTCCGTCAGGACTACAAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_176,NT_176,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGTCGATAACCACCATTTTTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_177,NT_177,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATAAATGGCTGCTGAGGGTAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_178,NT_178,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GCAAATGATCATGTCTCGAAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_179,NT_179,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTTTGTAAATACTATTGTGCA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_180,NT_180,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TATGAGAACCGTGTTATTCTG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_181,NT_181,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACTATTTACCACCGACGTGTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_182,NT_182,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTTGATGCACGCGAATACGCT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_183,NT_183,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGTCCCCGATGGGGCTGAATG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_184,NT_184,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TACTGTTGATCCCAATTGCGA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_185,NT_185,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGTATTGAAGGTGCAATTATT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_186,NT_186,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGTATTTTGTACGGGTCCACC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_187,NT_187,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTCATTCGCTCTTTTTAAGGG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_188,NT_188,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGAACCAAAATCTCACACCGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_189,NT_189,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGGACTTCATGAATACATCGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_190,NT_190,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACCATCCGCCTCTCCCATATT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_191,NT_191,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGACTGCGCTGAGATAAACGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_192,NT_192,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGAGTACCAGGCGGTTGCTGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_193,NT_193,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGTGGGAGTTGGTTCGTATAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_194,NT_194,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TCTGGTGCTAGCGTAAGTCTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_195,NT_195,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TAGACTTACTGCTACTTCCGG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_196,NT_196,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGTTCATCGTAGCTTCTCAGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_197,NT_197,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GATGGTATCCGAACCTTCTAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_198,NT_198,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTGATTACATTACGCCCGCAC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_199,NT_199,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTAGCACGCGATGTGACATA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_200,NT_200,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTCGCTCGATTGTGACGTTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_201,NT_201,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATAGATACGTGTAGATCTTCG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_202,NT_202,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GCTATTCTCTATTTTTGTTTG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_203,NT_203,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CCTGGTGCAAGATTTGAGGGG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_204,NT_204,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CGCGTCACGCTTGATGGGTTA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_205,NT_205,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGGATTCGTTATAAATACGAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_206,NT_206,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGCACTTTCGGTTTCACGTAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_207,NT_207,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTACCGTGATCGAAAGTATCC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_208,NT_208,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACTATATACAGCTTGGCCCTC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_209,NT_209,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGGGATCAGTGTTTCGTAAAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_210,NT_210,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAGACAAATTTGGGAAATTCT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_211,NT_211,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CCTAGGGTCGGTTGTATCCAC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_212,NT_212,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATTAGTGACTACTTGGCAGGA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_213,NT_213,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TACGTGCTCTTTCAAGGGATA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_214,NT_214,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAGCGAAAACTTGGTCCCAAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_215,NT_215,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTTTGTTCCTGGGACGAACCC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_216,NT_216,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATCCGTGACCAATTTAATAAC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_217,NT_217,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGGGCCGCACGGTAAGTATAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_218,NT_218,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTTTATACTGCAGCAAAATTC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_219,NT_219,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTACGTTCTAATGGGTTACCA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_220,NT_220,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CGCTTATCGTGAACAGCATAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_221,NT_221,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATTGTGATCATTTAATAAATT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_222,NT_222,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TCTAATTACTGCGGATCAATA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_223,NT_223,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTCGCATTTCATATGACAAAG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_224,NT_224,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTAGGCGGACGTTGAGTAAAC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_225,NT_225,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGCATTTCAGGTATCTAGCTC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_226,NT_226,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGTGCGATCCTCTGAGAAAG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_227,NT_227,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TCGACTAACTTGGGTCTCATC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_228,NT_228,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ACGAGACCATATGGGGCACCC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_229,NT_229,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AATAAATACAAGTGACTACAG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_230,NT_230,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTTGTCAACTTCGTTGGATCA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_231,NT_231,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATGATATTTTTAGTTCACATG,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_232,NT_232,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATTGGATTACTTGGTATATGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_233,NT_233,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTGACATATTTTTTATTTCAT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_234,NT_234,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GATCTTCCACCGGCGTGACGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_235,NT_235,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TCTATAAACATTAATGCTAGA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_236,NT_236,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GTTATACGCGTCAGCTGCATC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_237,NT_237,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CTTGTATGTACATTGTTACTC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_238,NT_238,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGCTATTTGCGAAAGGAATT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_239,NT_239,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TCATGCTTATTATGAGAGCGA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_240,NT_240,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AAACTTCTTGCTTACCCCCCA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_241,NT_241,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GGGAACTTATTGGTTGTAGGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_242,NT_242,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATGGGAGGAGTTTCTATTGGT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_243,NT_243,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),GAGTCAAGGTACAAAAGAATA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_244,NT_244,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TTGAGTATATATGGGTGTATC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_245,NT_245,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATTCGCTTTACCTGAATGCTT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_246,NT_246,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CACCTCGACTCTCTTTCAATT,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_247,NT_247,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),ATTCCTCTTTAATAGGTTAGC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_248,NT_248,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),TGCCCTTTATTCGTGTTCGAA,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_249,NT_249,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),AGGTTAAAAGTGAGGAGGTAC,CRISPR Guide Capture,Non-Targeting,Non-Targeting
NT_250,NT_250,R2,5PNNNNNNNNNNNNNNNNNNNN(BC),CGGCAAAACTCATGACTCACT,CRISPR Guide Capture,Non-Targeting,Non-Targeting

In [9]:
%%bash
mkdir -p /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/count_2/
cd /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/count_2/
sbatch \
    -p new,all \
    -o /data/reddylab/Alex/collab/20210526_Sean/logs/gex_10x_processing.v2.out \
    --mem=64G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/reddylab/software/cellranger/cellranger-6.0.1/bin/cellranger count \
    --id=gene_exp_guide \
    --expect-cells=20000 \
    --target-panel=/data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/immunology_v1.0_GRCh38-2020-A.target_panel.csv \
    --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
    --feature-ref=/data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/CD2_guide_feature_reference.csv  \
    --libraries=/data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/gene_exp_guide_lib.csv \
    --localcores=24 \
    --localmem=64
EOF

Submitted batch job 25434175


In [6]:
def revcomp(s):
    rev = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}
    return ''.join([rev[c] for c in s[::-1]])
revcomp("ATTCGCTTTACCTGAATGCTT")

'AAGCATTCAGGTAAAGCGAAT'

In [4]:
%%bash
mkdir -p /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/count/
cd /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/count/
#-15,17-20%10
SAMPLES=($(cut -f2 -d, '/data/reddylab/Alex/tmp/cell_ranger.clean.csv' | tail -n+2))
# 1-$((${#SAMPLES[@]}-1))
sbatch \
    --array=0 \
    -p new,all \
    -o /data/reddylab/Alex/collab/20210526_Sean/logs/sgrnas_10x_processing.%a.out \
    --mem=64G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
SAMPLES=($(cut -f2 -d, '/data/reddylab/Alex/tmp/cell_ranger.clean.csv' | tail -n+2))
SAMPLE=${SAMPLES[${SLURM_ARRAY_TASK_ID}]}
/data/reddylab/software/cellranger/cellranger-6.0.1/bin/cellranger count \
    --id=${SAMPLE} \
    --fastqs=/data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/fastq \
    --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
    --feature-ref=/data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/feature_reference.csv \
    --libraries=/data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/feature_barcode_lib.csv \
    --sample=${SAMPLE} \
    --localcores=24 \
    --localmem=64 \
    --chemistry=SC5P-R2
EOF

Submitted batch job 25434148


Create protospacers index

In [1]:
import numpy as np

import pandas as pd
target_guides = pd.read_csv("/data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/CD2_guide_feature_reference.csv")
with open("/data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/CD2_guide_protospacers.fasta", 'w') as ofile:
    for ii in target_guides.iterrows():
        ofile.write(">%s\n" % ii[1]["id"])
        ofile.write("%s\n" % ii[1]["sequence"])
        

Create Bowtie2 index

In [20]:
%%bash
module load bowtie2
sbatch \
    -o /data/reddylab/Alex/collab/20210526_Sean/logs/sgrnas.bt2_index.out \
    -pnew,all \
    --mem 16G \
    --cpus-per-task 8 \
    <<'EOF'
#!/bin/bash
bowtie2-build \
    --threads 8 \
    --seed 17 \
    /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/CD2_guide_protospacers.revcomp.fasta \
    /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/CD2_guide_protospacers.revcomp.bt2_ix

EOF

Submitted batch job 25436599


In [1]:
len("NNNNNNNNNNNNNNNNNNNN")

20

In [2]:
%%bash
module load bowtie2
module load samtools
sbatch -pall \
    --mem 32G \
    --cpus-per-task 8 \
    --array=1 \
    --output /data/reddylab/Alex/collab/20210526_Sean/logs/bowtie2.align_unaligned_reads_to_custom_index.%a.out \
    <<'EOF'
#!/bin/bash
# if [[ ! -e /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/count/Guide_Lib_${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.fastq.gz ]];
# then
#     samtools bam2fq \
#         --threads 8 \
#         -c 1 \
#         -0 /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/count/Guide_Lib_${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.fastq.gz \
#         -f 4 \
#         /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/count/Guide_Lib_${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.bam
# fi
bowtie2 \
    --trim3 20 \
    --trim5 49 \
    --no-unal \
    --norc \
    --end-to-end \
    -D 15 -R 2 -N 1 -L 18 -i S,1,0 \
    --score-min G,0,0 \
    --ignore-quals \
    -x /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/metadata/CD2_guide_protospacers.revcomp.bt2_ix \
    -U /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/count/Guide_Lib_${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.fastq.gz \
    --threads 8 \
| samtools view -b \
> /data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/count/Guide_Lib_${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.bt2.bam
EOF


Submitted batch job 27553552


In [3]:
len("CGACGCTCTTCCGATCTGATG")

21

In [2]:
len('GACGCTCTTCCGATCTCACAGGCAGGGCTTGATGAGATGTAATTTTCTTATATGGGTGCATATGACTGGCAGGAAAGACAGGGTAACTTT')-20-49

21

In [3]:
len("NNNNNNNNNNNNNNNNNNNN")

20

In [4]:
len("CGACGCTCTTCCGATCT")

17

In [7]:
len("ACTGCTCTCACAGTACTGCCA")

21

In [5]:
len("CGACGCTCTTCCGATCTCCTACACTCTTATCTGCGGTCCTATTTTTCTTATATGGGGATAATGTGCTGGCAGGAAAGACAAAACCGTAAG")

90

In [15]:
len("ACCGTATTTCCTCGCTCGTAC")

21

In [13]:
len("TGTTTCCAGAGTACTAAAAC")

20

In [19]:
len("CCCCATATAAGAAAACCTGTATTCCATATTGCTCAACGTTAGATCGGAA")

49

In [2]:
def revcomp(s):
    rev = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'N': 'N'}
    return ''.join([rev[c] for c in s[::-1]])


In [16]:
with open("/data/reddylab/software/spectre/foo.fa", "r") as ifile:
    with open("/data/reddylab/software/spectre/foo.revcom.fa", "w") as ofile:
        for l in ifile:
            ofile.write("%s\n" % revcomp(l.rstrip()))
        